In [2]:
# Importar las librerías
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
# Importar el dataset
movies = pd.read_csv("ml-1m/movies.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users  = pd.read_csv("ml-1m/users.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings  = pd.read_csv("ml-1m/ratings.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [4]:
# Preparar el conjunto de entrenamiento y elconjunto de testing
training_set = pd.read_csv("ml-100k/u1.base", sep = "\t", header = None)
training_set = np.array(training_set, dtype = "int")
test_set = pd.read_csv("ml-100k/u1.test", sep = "\t", header = None)
test_set = np.array(test_set, dtype = "int")

In [5]:
# Obtener el número de usuarios y de películas
nb_users = int(max(max(training_set[:, 0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [6]:
# Convertir los datos en un array X[u,i] con usuarios u en fila y películas i en columna
def convert(data):
    new_data = []
    for id_user in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_user]
        id_ratings = data[:, 2][data[:, 0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

In [7]:
# Convertir los datos a tensores de Torch
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [8]:
# Convertir las valoraciones a valores binarios 1 (Me gusta) o 0 (No me gusta)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1


In [9]:
# Crear la arquitectura de la Red Neuronal (Modelo Probabilistico Gráfico)
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):           #x = mini_batch_size x nv
        wx = torch.mm(x, self.W.t()) #mini_batch_size x nh
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):           #y = mini_batch_size x nh
        wy = torch.mm(y, self.W) #mini_batch_size x nv
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)   
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
        
nv = len(training_set[0])
nh = 100
batch_size = 100

rbm = RBM(nv, nh)


In [10]:
# Entrenar la RBM
nb_epoch = 10
for epoch in range(1, nb_epoch+1):
    training_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        training_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print("Epoch: "+str(epoch)+", Loss: "+str(training_loss/s))


Epoch: 1, Loss: tensor(0.3496)
Epoch: 2, Loss: tensor(0.2459)
Epoch: 3, Loss: tensor(0.2458)
Epoch: 4, Loss: tensor(0.2436)
Epoch: 5, Loss: tensor(0.2496)
Epoch: 6, Loss: tensor(0.2489)
Epoch: 7, Loss: tensor(0.2470)
Epoch: 8, Loss: tensor(0.2467)
Epoch: 9, Loss: tensor(0.2496)
Epoch: 10, Loss: tensor(0.2490)


In [11]:
# Testear la RBM
testing_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        testing_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
        print("Testing Loss: "+str(testing_loss/s))

Testing Loss: tensor(0.2701)
Testing Loss: tensor(0.2487)
Testing Loss: tensor(0.3196)
Testing Loss: tensor(0.2897)
Testing Loss: tensor(0.2984)
Testing Loss: tensor(0.2784)
Testing Loss: tensor(0.2649)
Testing Loss: tensor(0.2491)
Testing Loss: tensor(0.2325)
Testing Loss: tensor(0.2270)
Testing Loss: tensor(0.2303)
Testing Loss: tensor(0.2304)
Testing Loss: tensor(0.2381)
Testing Loss: tensor(0.2399)
Testing Loss: tensor(0.2572)
Testing Loss: tensor(0.2526)
Testing Loss: tensor(0.2573)
Testing Loss: tensor(0.2534)
Testing Loss: tensor(0.2453)
Testing Loss: tensor(0.2535)
Testing Loss: tensor(0.2579)
Testing Loss: tensor(0.2603)
Testing Loss: tensor(0.2545)
Testing Loss: tensor(0.2500)
Testing Loss: tensor(0.2444)
Testing Loss: tensor(0.2453)
Testing Loss: tensor(0.2560)
Testing Loss: tensor(0.2495)
Testing Loss: tensor(0.2510)
Testing Loss: tensor(0.2556)
Testing Loss: tensor(0.2573)
Testing Loss: tensor(0.2575)
Testing Loss: tensor(0.2497)
Testing Loss: tensor(0.2512)
Testing Loss: 

Testing Loss: tensor(0.2520)
Testing Loss: tensor(0.2519)
Testing Loss: tensor(0.2516)
Testing Loss: tensor(0.2517)
Testing Loss: tensor(0.2516)
Testing Loss: tensor(0.2510)
Testing Loss: tensor(0.2512)
Testing Loss: tensor(0.2512)
Testing Loss: tensor(0.2512)
Testing Loss: tensor(0.2514)
Testing Loss: tensor(0.2514)
Testing Loss: tensor(0.2512)
Testing Loss: tensor(0.2511)
Testing Loss: tensor(0.2517)
Testing Loss: tensor(0.2511)
Testing Loss: tensor(0.2520)
Testing Loss: tensor(0.2525)
Testing Loss: tensor(0.2520)
Testing Loss: tensor(0.2528)
Testing Loss: tensor(0.2545)
Testing Loss: tensor(0.2545)
Testing Loss: tensor(0.2539)
Testing Loss: tensor(0.2536)
Testing Loss: tensor(0.2535)
Testing Loss: tensor(0.2545)
Testing Loss: tensor(0.2548)
Testing Loss: tensor(0.2549)
Testing Loss: tensor(0.2549)
Testing Loss: tensor(0.2544)
Testing Loss: tensor(0.2555)
Testing Loss: tensor(0.2554)
Testing Loss: tensor(0.2549)
Testing Loss: tensor(0.2565)
Testing Loss: tensor(0.2560)
Testing Loss: 